In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
     |████████████████████████████████| 1.5MB 1.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.9-py3-none-any.whl size=73265 sha256=f26ffb0c2bd06f0eb3a0b17ce375365c4c6afb34b6dec1d695225ebd047d4e2f
  Stored in directory: /root/.cache/pip/wheels/9d/50/3d/2644504bb1e8c782f3fef5984f03d76fc4a74698fdec128b29
Successfully built kaggle


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c 2020-ml-p4

  0% 0.00/2.29M [00:00<?, ?B/s]
100% 2.29M/2.29M [00:00<00:00, 119MB/s]


In [5]:
!unzip 2020-ml-p4.zip

Archive:  2020-ml-p4.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import pandas as pd
import numpy as np

In [25]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('sample_submission.csv')
train[:5]

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,voted
0,2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,30s,3,1,3,Male,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1
1,3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,20s,4,2,0,Female,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,1
2,5,2.0,834,1.0,1769,4.0,2105,1.0,1070,5.0,1930,2.0,1258,5.0,1451,1.0,1029,3.0,1633,1.0,2254,5.0,1984,1.0,1158,3.0,1194,3.0,483,1.0,983,1.0,817,5.0,1052,1.0,1354,1.0,747,2.0,1826,40s,2,1,6,Female,1,3,White,Other,5,5,5,4,5,3,3,7,5,3,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2
3,8,5.0,795,2.0,3469,4.0,1693,3.0,1991,4.0,5922,2.0,1322,5.0,1014,5.0,3014,3.0,3771,3.0,13295,5.0,3605,3.0,9265,4.0,2531,1.0,812,4.0,2595,5.0,1227,5.0,1837,4.0,1218,4.0,9923,5.0,1631,20s,4,2,0,Female,2,1,Other,Christian_Other,1,1,2,1,2,2,2,2,2,3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1
4,9,2.0,1668,1.0,866,1.0,895,1.0,1308,2.0,1715,5.0,563,5.0,574,1.0,1406,5.0,1698,3.0,1354,5.0,774,1.0,1299,1.0,810,5.0,327,1.0,646,1.0,957,4.0,850,5.0,693,1.0,1313,1.0,986,50s,3,1,3,Female,1,1,White,Christian_Other,0,1,1,2,0,3,0,4,1,5,2,0,0,1,1,1,0,1,1,0,1,1,0,1,1,1,1,1


In [ ]:
train.info()

---
##### 나이 바꾸기

In [9]:
train['age_group'].unique()

array(['30s', '20s', '40s', '50s', '10s', '60s', '+70s'], dtype=object)

In [26]:
age = {'30s':3, '20s':2, '40s':4, '50s':5, '10s':1, '60s':6, '+70s':7}
train['age_group'] = train['age_group'].map(age)
test['age_group'] = test['age_group'].map(age)

train[:5]

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,voted
0,2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,3,3,1,3,Male,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1
1,3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,2,4,2,0,Female,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,1
2,5,2.0,834,1.0,1769,4.0,2105,1.0,1070,5.0,1930,2.0,1258,5.0,1451,1.0,1029,3.0,1633,1.0,2254,5.0,1984,1.0,1158,3.0,1194,3.0,483,1.0,983,1.0,817,5.0,1052,1.0,1354,1.0,747,2.0,1826,4,2,1,6,Female,1,3,White,Other,5,5,5,4,5,3,3,7,5,3,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2
3,8,5.0,795,2.0,3469,4.0,1693,3.0,1991,4.0,5922,2.0,1322,5.0,1014,5.0,3014,3.0,3771,3.0,13295,5.0,3605,3.0,9265,4.0,2531,1.0,812,4.0,2595,5.0,1227,5.0,1837,4.0,1218,4.0,9923,5.0,1631,2,4,2,0,Female,2,1,Other,Christian_Other,1,1,2,1,2,2,2,2,2,3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1
4,9,2.0,1668,1.0,866,1.0,895,1.0,1308,2.0,1715,5.0,563,5.0,574,1.0,1406,5.0,1698,3.0,1354,5.0,774,1.0,1299,1.0,810,5.0,327,1.0,646,1.0,957,4.0,850,5.0,693,1.0,1313,1.0,986,5,3,1,3,Female,1,1,White,Christian_Other,0,1,1,2,0,3,0,4,1,5,2,0,0,1,1,1,0,1,1,0,1,1,0,1,1,1,1,1


---
##### 성별바꾸기

In [11]:
train['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [27]:
sex = {'Male':0, 'Female':1}
train['gender'] = train['gender'].map(sex)
test['gender'] = test['gender'].map(sex)
train[:5]

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,voted
0,2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,3,3,1,3,0,1,2,White,Other,2,3,1,5,3,4,2,6,1,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1
1,3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,2,4,2,0,1,1,1,Asian,Hindu,2,4,1,1,1,3,1,3,1,3,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,1
2,5,2.0,834,1.0,1769,4.0,2105,1.0,1070,5.0,1930,2.0,1258,5.0,1451,1.0,1029,3.0,1633,1.0,2254,5.0,1984,1.0,1158,3.0,1194,3.0,483,1.0,983,1.0,817,5.0,1052,1.0,1354,1.0,747,2.0,1826,4,2,1,6,1,1,3,White,Other,5,5,5,4,5,3,3,7,5,3,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2
3,8,5.0,795,2.0,3469,4.0,1693,3.0,1991,4.0,5922,2.0,1322,5.0,1014,5.0,3014,3.0,3771,3.0,13295,5.0,3605,3.0,9265,4.0,2531,1.0,812,4.0,2595,5.0,1227,5.0,1837,4.0,1218,4.0,9923,5.0,1631,2,4,2,0,1,2,1,Other,Christian_Other,1,1,2,1,2,2,2,2,2,3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1
4,9,2.0,1668,1.0,866,1.0,895,1.0,1308,2.0,1715,5.0,563,5.0,574,1.0,1406,5.0,1698,3.0,1354,5.0,774,1.0,1299,1.0,810,5.0,327,1.0,646,1.0,957,4.0,850,5.0,693,1.0,1313,1.0,986,5,3,1,3,1,1,1,White,Christian_Other,0,1,1,2,0,3,0,4,1,5,2,0,0,1,1,1,0,1,1,0,1,1,0,1,1,1,1,1


---
##### 인종 & 종교 바꾸기

In [14]:
print(train['race'].unique())
print(train['religion'].unique())

['White' 'Asian' 'Other' 'Black' 'Native American' 'Arab'
 'Indigenous Australian']
['Other' 'Hindu' 'Christian_Other' 'Atheist' 'Agnostic'
 'Christian_Catholic' 'Christian_Protestant' 'Buddhist' 'Jewish' 'Muslim'
 'Christian_Mormon' 'Sikh']


In [28]:
race = {'White':1, 'Asian':2, 'Other':7, 'Black':3, 'Native American':4, 'Arab':5,'Indigenous Australian':6}
religion = {'Other':12, 'Hindu':1, 'Christian_Other':2, 'Atheist':3, 'Agnostic':4,'Christian_Catholic':5,'Christian_Protestant':6, 'Buddhist':7,
            'Jewish':8, 'Muslim':9,'Christian_Mormon':10, 'Sikh':11}
train['race'] = train['race'].map(race)
train['religion'] = train['religion'].map(religion)

test['race'] = test['race'].map(race)
test['religion'] = test['religion'].map(religion)

---

In [29]:
x_train = train.loc[:,'QaA':'wr_13']
y_train = train.loc[:,'voted']

x_test = test.loc[:,'QaA':'wr_13']

In [30]:
x_train[:5]

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,3,3,1,3,0,1,2,1,12,2,3,1,5,3,4,2,6,1,3,2,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1
1,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,2,4,2,0,1,1,1,2,1,2,4,1,1,1,3,1,3,1,3,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1
2,2.0,834,1.0,1769,4.0,2105,1.0,1070,5.0,1930,2.0,1258,5.0,1451,1.0,1029,3.0,1633,1.0,2254,5.0,1984,1.0,1158,3.0,1194,3.0,483,1.0,983,1.0,817,5.0,1052,1.0,1354,1.0,747,2.0,1826,4,2,1,6,1,1,3,1,12,5,5,5,4,5,3,3,7,5,3,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,5.0,795,2.0,3469,4.0,1693,3.0,1991,4.0,5922,2.0,1322,5.0,1014,5.0,3014,3.0,3771,3.0,13295,5.0,3605,3.0,9265,4.0,2531,1.0,812,4.0,2595,5.0,1227,5.0,1837,4.0,1218,4.0,9923,5.0,1631,2,4,2,0,1,2,1,7,2,1,1,2,1,2,2,2,2,2,3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1
4,2.0,1668,1.0,866,1.0,895,1.0,1308,2.0,1715,5.0,563,5.0,574,1.0,1406,5.0,1698,3.0,1354,5.0,774,1.0,1299,1.0,810,5.0,327,1.0,646,1.0,957,4.0,850,5.0,693,1.0,1313,1.0,986,5,3,1,3,1,1,1,1,2,0,1,1,2,0,3,0,4,1,5,2,0,0,1,1,1,0,1,1,0,1,1,0,1,1,1,1


In [31]:
from sklearn.preprocessing import StandardScaler, Normalizer
std = StandardScaler()
nor = Normalizer()

x_train_std = std.fit_transform(x_train)
x_test_std = std.fit_transform(x_test)
x_train_nor = nor.fit_transform(x_train)
x_test_nor = nor.fit_transform(x_test)

In [32]:
from collections import Counter
print(Counter(y_train))

Counter({2: 12277, 1: 10033})


In [33]:
submit[:5]

,index,voted
0,6,1
1,10,1
2,12,1
3,14,1
4,25,1


In [36]:
'''
#KNN 
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=200,p=2,weights='distance')
KNN.fit(x_train_nor,y_train)

test_pred = KNN.predict(x_test_nor)

for i in range(len(test_pred)):
  submit['voted'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
!kaggle competitions submit -c 2020-ml-p4 -f submission.csv -m "KNN"
'''

100% 72.5k/72.5k [00:09<00:00, 7.95kB/s]
Successfully submitted to 2020.ML.중간고사.문제4

In [41]:
# 원본 + gini + max_depth = 5 / 0.68
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=1,criterion='gini',max_depth=5)

dtc.fit(x_train,y_train)
test_pred = dtc.predict(x_test)

for i in range(len(test_pred)):
  submit['voted'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index = None)
!kaggle competitions submit -c 2020-ml-p4 -f submission.csv -m "dtc"

100% 72.5k/72.5k [00:06<00:00, 12.1kB/s]
Successfully submitted to 2020.ML.중간고사.문제4